In [1]:
import tensorflow as tf
from tensorflow.python.framework import ops
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
from numpy.random import rand, choice
npop = 100
nsets = 10
Cmat = (rand(nsets,npop) > rand(nsets, 1)) + 0.0

In [7]:
csizes = np.sum(Cmat, 1)

In [8]:
nsub = 30
csizes_scale = (csizes * nsub/npop)
csizes_scale = csizes_scale.reshape((-1, 1))
csizes_scale.shape

(10, 1)

In [45]:
nrestarts = 2000
ops.reset_default_graph()
lam_bin = tf.placeholder(dtype = tf.float32, shape = ())
lam_count = tf.placeholder(dtype = tf.float32, shape = ())
lam_loss = tf.placeholder(dtype = tf.float32, shape = ())
pre_x = tf.get_variable(dtype = tf.float32, shape = (npop, nrestarts), name = 'pre_x',
                     initializer = tf.random_normal_initializer(mean = 0.0, stddev = 0.1))
x = tf.nn.sigmoid(pre_x * 200)
C = tf.constant(Cmat, dtype = tf.float32)
cs = tf.constant(csizes_scale, dtype = tf.float32)
#pen_bin = tf.reduce_sum((.25 - (.5 - x)**2)**2)
pen_bin = tf.reduce_sum(tf.math.minimum(tf.math.abs(x), tf.math.abs(x - 1)))
pen_count = tf.reduce_sum((tf.reduce_sum(x, axis=0) - nsub)**2)
#pen_count = tf.reduce_sum(tf.math.maximum(0.0, tf.reduce_sum(x, axis=0) - nsub))
loss = tf.reduce_sum((tf.matmul(C, x) - cs)**2)
#objf = lam_loss * loss + lam_bin * pen_bin + lam_count * pen_count
objf = loss + lam_count * pen_count

In [46]:
opt = tf.train.AdamOptimizer(learning_rate = 0.001, beta1=0.9, beta2=0.999, epsilon=1e-8)
grads = opt.compute_gradients(objf)
apply_gradient_op = opt.apply_gradients(grads)

In [47]:
nits = 14000
with tf.Session() as sess:
    tf.global_variables_initializer().run(session=sess)
    for it_no in range(nits):
        #lam_bin_v = 100 * np.power(30 + it_no, 0.7)
        lam_count_v = 100  #100 * np.power(1 + it_no, 0.5)
        #lam_loss_v = 0 # 200/(200 + it_no)
        l, o, pb, pc, _ = sess.run([loss, objf, pen_bin, pen_count, apply_gradient_op], 
                           feed_dict={lam_count:lam_count_v})
        if it_no % 1000 == 0:
            print({'objf':o, 'loss':l, 'pen_bin': pb, 'pen_count': pc})
    x_found = sess.run(x)

{'objf': 89140380.0, 'loss': 3155945.8, 'pen_bin': 5567.518, 'pen_count': 859844.44}
{'objf': 49912.445, 'loss': 39265.062, 'pen_bin': 2617.5278, 'pen_count': 106.47385}
{'objf': 35237.72, 'loss': 24776.02, 'pen_bin': 3074.8633, 'pen_count': 104.61699}
{'objf': 26261.66, 'loss': 15859.875, 'pen_bin': 3613.3745, 'pen_count': 104.01784}
{'objf': 20927.527, 'loss': 10539.716, 'pen_bin': 4203.415, 'pen_count': 103.87811}
{'objf': 18617.16, 'loss': 8243.685, 'pen_bin': 4703.524, 'pen_count': 103.73474}
{'objf': 17746.064, 'loss': 7377.1523, 'pen_bin': 5071.0635, 'pen_count': 103.689125}
{'objf': 17410.164, 'loss': 7043.3613, 'pen_bin': 5266.9243, 'pen_count': 103.668015}
{'objf': 17282.848, 'loss': 6913.9893, 'pen_bin': 5334.2705, 'pen_count': 103.68858}
{'objf': 17234.285, 'loss': 6859.815, 'pen_bin': 5316.8784, 'pen_count': 103.744705}
{'objf': 17204.602, 'loss': 6834.5205, 'pen_bin': 5244.208, 'pen_count': 103.700806}
{'objf': 17196.777, 'loss': 6823.407, 'pen_bin': 5139.0786, 'pen_count

In [48]:
x_round = (x_found > 0.5) + 0.0

In [50]:
#plt.matshow(np.concatenate((csizes_scale, np.matmul(Cmat, x_round)), axis=1))

In [52]:
losses = np.sum((np.matmul(Cmat, x_round) - csizes_scale)**2, axis = 0)
np.sort(losses[np.sum(x_round, 0) == nsub])[:10]

array([1.18, 1.18, 1.18, 1.18, 1.18, 1.18, 1.18, 1.18, 1.18, 1.18])

In [53]:
from numpy.random import choice
nrestarts = 1000000
x0 = np.zeros((npop, nrestarts))
for i in range(nrestarts):
    x0[choice(npop, nsub, False), i] = 1

In [54]:
losses_rand = np.sum((np.matmul(Cmat, x0) - csizes_scale)**2, axis = 0)

In [55]:
np.sort(losses_rand)

array([  1.38,   1.58,   1.78, ..., 178.98, 179.98, 182.58])